In [10]:
%%time
%reload_ext autoreload
%autoreload
%autoreload 2
%config Completer.use_jedi = False

import os
import json
import warnings
import pprint
import sys
import numpy as np
import pandas as pd
import torch

MAIN_PATH = os.getcwd().split("notebooks")[0]
sys.path.insert(0, MAIN_PATH)

CPU times: user 45.6 ms, sys: 35.2 ms, total: 80.7 ms
Wall time: 81.3 ms


In [11]:
# Run a Federated Learning experiment
from data_loader.mnist import MnistDatasetManager
from models.mnist_model import MnistFullConnectModel, SimpleCNN
from server.base_server import BaseServer
from client.base_client import BaseClient
from experiments.base_experiment import BaseExperiment
from gradients.noise import GaussianNoiseGenerator, NoNoiseGenerator,StaircaseNoiseGenerator
from metrics.classification import multiclass_accuracy

In [12]:
class DemoMnistExperiment(BaseExperiment):
    def __init__(self, 
                 client_num: int = 2, 
                 lr: float=0.01, 
                 noise_generator=None,
                 max_norm = 3,
                 sampling_rate=0.01):
        if noise_generator is None:
            noise_generator = NoNoiseGenerator()
        self.noise_generator = noise_generator
        self.lr = lr
        self.max_norm = max_norm
        self.sampling_rate = sampling_rate
        self.client_num = client_num
        self._init_server_clients(client_num, self.lr)
        self._init_data(client_num)

    def _init_server_clients(self, client_num, lr):
        model = SimpleCNN
        self.clients = [BaseClient(model(lr=lr), 
                                   client_id=idx, 
                                   noise_generator=self.noise_generator)
                        for idx in range(client_num)]
        self.server = BaseServer(model(lr=lr, 
                                       max_norm=self.max_norm))

    def _init_data(self, client_num):
        data_manager = MnistDatasetManager(n_parties=client_num, 
                                           sampling_lot_rate=self.sampling_rate)
        self.client_train_datas = data_manager.train_loaders
        self.valid_datas = data_manager.validation_loader
        self.test_data = data_manager.test_loader

    def evaluate_model(self, data):
        total_correct = 0
        total_sample_num = 0
        with torch.no_grad():
            for _, (inputs, target) in enumerate(data):
                predict_labels = self.server.predict(inputs)
                correct, sample_num = multiclass_accuracy(y_pred=predict_labels, 
                                                          y_true=target)
                total_correct += correct
                total_sample_num += sample_num
                
        return total_correct / total_sample_num
        
    def get_validation_result(self):
        return self.evaluate_model(self.valid_datas)
    
    def get_test_result(self):
        return self.evaluate_model(self.test_data)
    
    def aggeragate(self):
        self.server.aggeragate_model(self.clients)
    
    def run(self, epochs: int):
        self._init_data(self.client_num)
        for client in self.clients:
            client.set_training_mode(for_gradient=False)

        for epoch in range(epochs):
            print(self.get_validation_result())
            for client, client_train_data in self.shuffled_data(to_shuffle=False):
                # client_train_data is a generator of dataloders
                client.train(client_train_data, client_epochs=2)

            self.aggeragate()
            if epoch and not (epoch % 1):
                print(self.get_validation_result())

            self.distribute_model()


In [17]:
EXPERIMENT = DemoMnistExperiment(client_num=1, 
                                 lr = 0.001, 
                                 noise_generator=NoNoiseGenerator())

In [19]:
EXPERIMENT.run(10)

0.4445
480
480
0.6450833333333333
480
480
0.6894166666666667
0.6894166666666667
480
480
0.7573333333333333
0.7573333333333333
480
480
